# Part I. ETL Pipeline for Pre-Processing the Files

Import Python packages

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
from cassandra.query import dict_factory
from tqdm import tqdm

Creating list of filepaths to process original event csv data files

In [3]:
# checking our current working directory
print(os.getcwd())

# get our current folder and subfolder event data
filepath = os.path.join(os.getcwd(), 'event_data')

# create a for loop to build a list of files and collect each filepath
for root, dirs, files in os.walk(filepath): 
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root, '*'))

print(file_path_list)

/home/eugene/PycharmProjects/cassandra-dm
['/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-28-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-06-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-26-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-01-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-25-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-11-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-04-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-09-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-18-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-15-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-13-events.csv', '/home/eugene/PycharmProjects/cassandra-dm/event_data/2018-11-16-events.csv', '/home/eugene/Pycharm

Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


Let's check the number of rows in our csv file

In [5]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))


6821


In [6]:
df = pd.read_csv('event_datafile_new.csv')

In [7]:
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Mitch Ryder & The Detroit Wheels,Tegan,F,65,Levine,205.03465,paid,"Portland-South Portland, ME",992,Jenny Take A Ride (LP Version),80
1,The Spill Canvas,Tegan,F,66,Levine,358.03383,paid,"Portland-South Portland, ME",992,The TIde (LP Version),80
2,Mogwai,Tegan,F,67,Levine,571.19302,paid,"Portland-South Portland, ME",992,Two Rights Make One Wrong,80
3,Spor,Tegan,F,68,Levine,380.34240,paid,"Portland-South Portland, ME",992,Way Of The Samurai,80
4,DJ Dizzy,Tegan,F,69,Levine,221.15220,paid,"Portland-South Portland, ME",992,Sexy Bitch,80


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- lastName of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

# Part II. Modeling Apache Cassandra database

Creating a Cluster

This should make a connection to a Cassandra instance our local machine (127.0.0.1)

In [8]:
from cassandra.cluster import Cluster
cluster = Cluster()

# to establish connection and begin executing queries, need a session
session = cluster.connect()

In [9]:
session.row_factory = dict_factory

Create Keyspace

In [10]:
session.execute("CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

Set Keyspace

In [11]:
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




Query 1:  Give me the artist, song title and song's length in the music app history that was heard during `sessionId = 338`, and `itemInSession = 4`

In [12]:
def create_table(table_name, schema, session=session):
    query = "CREATE TABLE IF NOT EXISTS {} {}".format(table_name, schema)
    try:
        session.execute(query)
        print('table {} is created...'.format(table_name))
    except Exception as e:
        print(e)

Let's create necessary tables

In [13]:
tables = [
    'music_app_history_1', 
    'music_app_history_2', 
    'music_app_history_3',
]

schemas = [
    '(sessionid INT, iteminsession INT, artist TEXT, title TEXT, length FLOAT, PRIMARY KEY (sessionid, iteminsession))',
    '(userid INT, sessionid INT, iteminsession INT, artist TEXT, title TEXT, user_first_name TEXT, user_last_name TEXT, \
    PRIMARY KEY ((userid, sessionid), iteminsession))',
    '(title TEXT, user_first_name TEXT, user_last_name TEXT, PRIMARY KEY ((title), user_first_name, user_last_name))',
]

for table_name, schema in zip(tables, schemas):
    create_table(table_name, schema)

table music_app_history_1 is created...
table music_app_history_2 is created...
table music_app_history_3 is created...


In [12]:
query = "CREATE TABLE IF NOT EXISTS {} {}".format(
'music_app_history', 
'(sessionId INT, itemInSession INT, artist TEXT, title TEXT, length FLOAT, PRIMARY KEY (sessionId, itemInSession))')

try:
    session.execute(query)
except Exception as e:
    print(e)

Insert data into tables

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        query = "INSERT INTO music_app_history (sessionid, iteminsession, artist, title, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

6820it [00:04, 1430.71it/s]


In [ ]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_app_history_2 (userid, sessionid, iteminsession, artist, title, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history_3 (title, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4]))

Verify the data have been inserted into each table

In [ ]:
def query_data(query, session=session):
    df = pd.DataFrame(list(session.execute(query)))
    return df

In [ ]:
query = "SELECT * FROM music_app_history WHERE sessionid = 338 AND iteminsession = 4"

result_df = query_data(query)

In [ ]:
result_df

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM music_app_history WHERE sessionid = 338 AND iteminsession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

In [ ]:
def dataframe_from_rows(rows):
    data = [dict(row) for row in rows]
    df = pd.DataFrame(data)
    df.head()

In [ ]:
for r in rows:
    print(r)

In [ ]:
df = pd.DataFrame(list(session.execute(query)))

In [ ]:
df

In [ ]:
data = (row for row in rows)

In [ ]:
list(data)

In [ ]:
row.__class__

In [ ]:
print(df)

In [ ]:
pd.DataFrame(list(rows))

In [ ]:
dataframe.query('sessionId == 338 and itemInSession == 4')[['sessionId', 'itemInSession', 'artist', 'song', 'length']]

In [ ]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS {} {}".format(
'music_app_history_2', 
'(userid INT, sessionid INT, iteminsession INT, artist TEXT, title TEXT, user_first_name TEXT, user_last_name TEXT, PRIMARY KEY ((userid, sessionid), iteminsession))')

try:
    session.execute(query)
except Exception as e:
    print(e)


In [ ]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history_3 (title, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4]))

Verify the data have been inserted into each table

In [49]:
def query_data(query, session=session):
    df = pd.DataFrame(list(session.execute(query)))
    return df

In [50]:
query = "SELECT * FROM music_app_history WHERE sessionid = 338 AND iteminsession = 4"

result_df = query_data(query)

In [51]:
result_df

,sessionid,iteminsession,artist,length,title
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


#### Do a SELECT to verify that the data have been inserted into each table

In [32]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM music_app_history WHERE sessionid = 338 AND iteminsession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

{'sessionid': 338, 'iteminsession': 4, 'artist': 'Faithless', 'length': 495.30731201171875, 'title': 'Music Matters (Mark Knight Dub)'}


In [20]:
def dataframe_from_rows(rows):
    data = [dict(row) for row in rows]
    df = pd.DataFrame(data)
    df.head()

In [33]:
for r in rows:
    print(r)

In [47]:
df = pd.DataFrame(list(session.execute(query)))

In [48]:
df

,sessionid,iteminsession,artist,length,title
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


In [44]:
data = (row for row in rows)

In [46]:
list(data)

[]

In [43]:
row.__class__

dict

In [25]:
print(df)

Empty DataFrame
Columns: []
Index: []


In [17]:
pd.DataFrame(list(rows))

""


In [15]:
dataframe.query('sessionId == 338 and itemInSession == 4')[['sessionId', 'itemInSession', 'artist', 'song', 'length']]

,sessionId,itemInSession,artist,song,length
5822,338,4,Faithless,Music Matters (Mark Knight Dub),495.3073


In [16]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS {} {}".format(
'music_app_history_2', 
'(userid INT, sessionid INT, iteminsession INT, artist TEXT, title TEXT, user_first_name TEXT, user_last_name TEXT, PRIMARY KEY ((userid, sessionid), iteminsession))')

try:
    session.execute(query)
except Exception as e:
    print(e)


In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history_2 (userid, sessionid, iteminsession, artist, title, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


In [18]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM music_app_history_2 WHERE userid = 10 AND sessionid = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

data = [dict(row) for row in rows]
df = pd.DataFrame(data)
df.head()

{'userid': 10, 'sessionid': 182, 'iteminsession': 0, 'artist': 'Down To The Bone', 'title': "Keep On Keepin' On", 'user_first_name': 'Sylvie', 'user_last_name': 'Cruz'}
{'userid': 10, 'sessionid': 182, 'iteminsession': 1, 'artist': 'Three Drives', 'title': 'Greece 2000', 'user_first_name': 'Sylvie', 'user_last_name': 'Cruz'}
{'userid': 10, 'sessionid': 182, 'iteminsession': 2, 'artist': 'Sebastien Tellier', 'title': 'Kilometer', 'user_first_name': 'Sylvie', 'user_last_name': 'Cruz'}
{'userid': 10, 'sessionid': 182, 'iteminsession': 3, 'artist': 'Lonnie Gordon', 'title': 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'user_first_name': 'Sylvie', 'user_last_name': 'Cruz'}


""


In [19]:
rows = session.execute(query)
data = [dict(row) for row in rows]

Let's check our data model for that query using initial data frame

In [20]:
dataframe.query('userId == 10 and sessionId == 182')[['userId', 'sessionId', 'itemInSession', 'artist', 'song', 'firstName', 'lastName']]

,userId,sessionId,itemInSession,artist,song,firstName,lastName
6368,10,182,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
6369,10,182,1,Three Drives,Greece 2000,Sylvie,Cruz
6370,10,182,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
6371,10,182,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [21]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS {} {}".format(
'music_app_history_3', 
'(title TEXT, user_first_name TEXT, user_last_name TEXT, PRIMARY KEY ((title), user_first_name, user_last_name))')

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history_3 (title, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4]))


In [23]:
query = "SELECT * FROM music_app_history_3 WHERE title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

{'title': 'All Hands Against His Own', 'user_first_name': 'Jacqueline', 'user_last_name': 'Lynch'}
{'title': 'All Hands Against His Own', 'user_first_name': 'Sara', 'user_last_name': 'Johnson'}
{'title': 'All Hands Against His Own', 'user_first_name': 'Tegan', 'user_last_name': 'Levine'}


Let's check query results using initial data frame

In [24]:
dataframe.query("song == 'All Hands Against His Own'")[['song', 'firstName', 'lastName']]

,song,firstName,lastName
1476,All Hands Against His Own,Tegan,Levine
3152,All Hands Against His Own,Sara,Johnson
4919,All Hands Against His Own,Jacqueline,Lynch


### Drop the tables before closing out the sessions

In [25]:
def drop_table(table_name):
    query = "DROP TABLE IF EXISTS {}".format(table_name)
    try:
        session.execute(query)
        print('table {} is dropped...'.format(table_name))
    except Exception as e:
        print(e)


In [26]:
drop_table('music_app_history_1')
drop_table('music_app_history_2')
drop_table('music_app_history_3')

table music_app_history_1 is dropped...
table music_app_history_2 is dropped...
table music_app_history_3 is dropped...


### Close the session and cluster connection

In [27]:
session.shutdown()
cluster.shutdown()